## Data evaluation for continuous ratings

Zizhuang Miao

This script is used to correlate the means/medians of online participants' ratings with experimenters' annotations to see how similar they are. Along the way of analysis, this script also generates and saves .csv files that include both the mean/medians of online participants' ratings and the (resampled) experimenters' annotations.

### Social interactions

In [2]:
import pandas as pd
import numpy as np
from os.path import join
import statsmodels.api as sm
from scipy.stats import ttest_ind

##### mean

In [3]:
dataDir = 'C:\\'
parDataDir = join(dataDir, 'continuous_social_1a2a3a4a_long')
manualDataDir = join(dataDir, 'manualAnnotations_social_consensus')
outputDir = 'C:\\'

# this is a list of subjects whose ratings are not like the average of other participants (correlation < 0.3 across all narratives)
# we will exclude them from generating group-level summary statistics
badSub = pd.read_csv('C:\\')['ID']

corr_narrative = []
regress_pRsquare_narrative = []
regress_z_narrative = []
t_narratvie = []
p_t_narrative = []

allData = pd.DataFrame()

for n in range(1,9):
    parData = pd.DataFrame()
    manualData = pd.DataFrame()
    lastTimepoint, lastTimepoint2 = [], []    # the last time points in each situation

    for s in range(1,10):
        # participants' ratings
        parDat = pd.read_csv(join(parDataDir, f"narrative{n}situation{s}.csv"))
        if s != 1:
            parDat['time'] += 0.23
            parDat['time'] += lastTimepoint[-1]
        lastTimepoint.append(parDat.loc[len(parDat)-1, 'time'])
        
        parDat = parDat[~parDat['ID'].isin(badSub)]    # excluding subjects that are not like the group
        
        parData = pd.concat([parData, parDat], ignore_index=True)
        parData = parData.drop(columns='index') if s!=1 else parData
        parData = parData.reset_index()

        # manual ratings
        manualDat = pd.read_csv(join(manualDataDir, f"narrative{n}situation{s}.csv"))
        if s != 1:
            manualDat['time'] += 0.23
            manualDat['time'] += lastTimepoint2[-1]
        lastTimepoint2.append(manualDat.loc[len(manualDat)-1, 'time'])
        manualDat['situation'] = s
        manualDat['validPar_trial'] = len(np.unique(parDat['ID']))     # the number of participants who have valid data in this trial
        
        nvalid_time = parDat.groupby('time').count()    # the number of participants who have valid data at each time point
        nvalid_time = nvalid_time.reset_index()
        manualDat['validPar_time'] = nvalid_time['rating']
        
        manualData = pd.concat([manualData, manualDat], ignore_index=True)
        manualData = manualData.drop(columns='index') if s!= 1 else manualData
        manualData = manualData.reset_index()

    manualData = manualData.drop(columns='index')
    manualData.columns = ['time', 'labels', 'situation', 'validPar_trial', 'validPar_time']
    manualData = manualData[['situation', 'time', 'labels', 'validPar_trial', 'validPar_time']]
    
    parData_mean = parData.groupby('time')['rating'].mean()
    parData_mean = parData_mean.reset_index()
    parData_mean = parData_mean.drop(columns='time')
    parData_mean.columns = ['mean']
    parData_std = parData.groupby('time')['rating'].std()
    parData_std = parData_std.reset_index()
    parData_std = parData_std.drop(columns='time')
    parData_std.columns = ['std']
    
    combinedData = pd.concat([manualData, parData_mean, parData_std], axis=1)
    combinedData.to_csv(join(dataDir, 'combined_social_consensus', f'narrative{n}_mean.csv'), index=False)
    allData = pd.concat([allData, combinedData], ignore_index=True)

    # calculate metrics
    # correlation
    corr_narrative.append(combinedData['labels'].corr(combinedData['mean']))
    # logistic regression
    binLabels = pd.DataFrame({'labels': [1 if x > 50 else 0 for x in combinedData['labels']]})
    X = combinedData['mean']
    binLabels = binLabels.loc[~X.isna(), 'labels']
    X = sm.add_constant(X)
    X = X.dropna()
    logit_model = sm.Logit(binLabels, X)
    result = logit_model.fit()
    regress_pRsquare_narrative.append(result.prsquared)
    regress_z_narrative.append(result.tvalues[1])
    # t test
    t_stat, p_val = ttest_ind(binLabels, X.iloc[:, 1])
    t_narratvie.append(t_stat)
    p_t_narrative.append(p_val)

# calculate metrics for all narratives concatenated
# correlation
corr_narrative.append(allData['labels'].corr(allData['mean']))
# logistic regression
binLabels = pd.DataFrame({'labels': [1 if x > 50 else 0 for x in allData['labels']]})
X = allData['mean']
binLabels = binLabels.loc[~X.isna(), 'labels']
X = sm.add_constant(X)
X = X.dropna()
logit_model = sm.Logit(binLabels, X)
result = logit_model.fit()
regress_pRsquare_narrative.append(result.prsquared)
regress_z_narrative.append(result.tvalues[1])
# t test
t_stat, p_val = ttest_ind(binLabels, X.iloc[:, 1])
t_narratvie.append(t_stat)
p_t_narrative.append(p_val)


outputDf_narrative = pd.DataFrame({'narrative': list(range(1,9)) + ['All'], 
                                   'pearson_r': corr_narrative, 't': t_narratvie, 'p_ttest': p_t_narrative,
                                   'z_logistic': regress_z_narrative, 'pseudo_r': regress_pRsquare_narrative})
outputDf_narrative = outputDf_narrative.round(3)
outputDf_narrative.to_csv(join(outputDir, 'similarity_social_consensus_mean.csv'), index=False)

Optimization terminated successfully.
         Current function value: 0.595862
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.340888
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.412228
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.504397
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.608431
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.439736
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.291491
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.554428
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.495364
         Iterations 6


##### median

In [4]:
from scipy.stats import median_abs_deviation

dataDir = 'C:\\'
parDataDir = join(dataDir, 'continuous_social_1a2a3a4a_long')
manualDataDir = join(dataDir, 'manualAnnotations_social_consensus')
outputDir = 'C:\\'

badSub = pd.read_csv('C:\\')['ID']

corr_narrative = []
regress_pRsquare_narrative = []
regress_z_narrative = []
t_narratvie = []
p_t_narrative = []

allData = pd.DataFrame()

for n in range(1,9):
    parData = pd.DataFrame()
    manualData = pd.DataFrame()
    lastTimepoint, lastTimepoint2 = [], []    # the last time points in each situation

    for s in range(1,10):
        # participants' ratings
        parDat = pd.read_csv(join(parDataDir, f"narrative{n}situation{s}.csv"))
        if s != 1:
            parDat['time'] += 0.23
            parDat['time'] += lastTimepoint[-1]
        lastTimepoint.append(parDat.loc[len(parDat)-1, 'time'])
        parDat = parDat[~parDat['ID'].isin(badSub)]    # excluding subjects that are not like the group
        parData = pd.concat([parData, parDat])
        parData = parData.drop(columns='index') if s!=1 else parData
        parData = parData.reset_index()

        # manual ratings
        manualDat = pd.read_csv(join(manualDataDir, f"narrative{n}situation{s}.csv"))
        if s != 1:
            manualDat['time'] += 0.23
            manualDat['time'] += lastTimepoint2[-1]
        lastTimepoint2.append(manualDat.loc[len(manualDat)-1, 'time'])
        manualDat['situation'] = s
        manualDat['validPar_trial'] = len(np.unique(parDat['ID']))     # the number of participants who have valid data in this trial
        nvalid_time = parDat.groupby('time').count()    # the number of participants who have valid data at each time point
        nvalid_time = nvalid_time.reset_index()
        manualDat['validPar_time'] = nvalid_time['rating']
        manualData = pd.concat([manualData, manualDat])
        manualData = manualData.drop(columns='index') if s!= 1 else manualData
        manualData = manualData.reset_index()

    manualData = manualData.drop(columns='index')
    manualData.columns = ['time', 'labels', 'situation', 'validPar_trial', 'validPar_time']
    manualData = manualData[['situation', 'time', 'labels', 'validPar_trial', 'validPar_time']]
    parData_median = parData.groupby('time')['rating'].median()
    parData_median = parData_median.reset_index()
    parData_median = parData_median.drop(columns='time')
    parData_median.columns = ['median']
    parData_mad = parData.dropna().groupby('time')['rating'].apply(median_abs_deviation)
    parData_mad = parData_mad.reset_index()
    parData_mad = parData_mad.drop(columns='time')
    parData_mad.columns = ['mad']
    combinedData = pd.concat([manualData, parData_median, parData_mad], axis=1)
    combinedData.to_csv(join(dataDir, 'combined_social_consensus', f'narrative{n}_median.csv'), index=False)
    allData = pd.concat([allData, combinedData], ignore_index=True)

    # calculate metrics
    # correlation
    corr_narrative.append(combinedData['labels'].corr(combinedData['median']))
    # logistic regression
    binLabels = pd.DataFrame({'labels': [1 if x > 50 else 0 for x in combinedData['labels']]})
    X = combinedData['median']
    binLabels = binLabels.loc[~X.isna(), 'labels']
    X = sm.add_constant(X)
    X = X.dropna()
    logit_model = sm.Logit(binLabels, X)
    result = logit_model.fit()
    regress_pRsquare_narrative.append(result.prsquared)
    regress_z_narrative.append(result.tvalues[1])
    # t test
    t_stat, p_val = ttest_ind(binLabels, X.iloc[:, 1])
    t_narratvie.append(t_stat)
    p_t_narrative.append(p_val)

# calculate metrics for all narratives concatenated
# correlation
corr_narrative.append(allData['labels'].corr(allData['median']))
# logistic regression
binLabels = pd.DataFrame({'labels': [1 if x > 50 else 0 for x in allData['labels']]})
X = allData['median']
binLabels = binLabels.loc[~X.isna(), 'labels']
X = sm.add_constant(X)
X = X.dropna()
logit_model = sm.Logit(binLabels, X)
result = logit_model.fit()
regress_pRsquare_narrative.append(result.prsquared)
regress_z_narrative.append(result.tvalues[1])
# t test
t_stat, p_val = ttest_ind(binLabels, X.iloc[:, 1])
t_narratvie.append(t_stat)
p_t_narrative.append(p_val)


outputDf_narrative = pd.DataFrame({'narrative': list(range(1,9)) + ['All'], 
                                   'pearson_r': corr_narrative, 't': t_narratvie, 'p_ttest': p_t_narrative,
                                   'z_logistic': regress_z_narrative, 'pseudo_r': regress_pRsquare_narrative})
outputDf_narrative = outputDf_narrative.round(3)
outputDf_narrative.to_csv(join(outputDir, 'similarity_social_consensus_median.csv'), index=False)

Optimization terminated successfully.
         Current function value: 0.593052
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.349076
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.426978
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.503582
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.611826
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.446155
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.296978
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.559985
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.498439
         Iterations 6


### Theory of mind

##### mean

In [6]:
dataDir = 'C:\\'
parDataDir = join(dataDir, 'continuous_tom_1a2a3a4a_long')
manualDataDir = join(dataDir, 'manualAnnotations_tom_zizhuang')
outputDir = 'C:\\'

# this is a list of subjects whose ratings are not like the average of other participants (correlation < 0.2 across all narratives)
# we will exclude them from generating group-level summary statistics
badSub = pd.read_csv('C:\\')['ID']

corr_narrative = []
regress_pRsquare_narrative = []
regress_z_narrative = []
t_narratvie = []
p_t_narrative = []

allData = pd.DataFrame()

for n in range(1,9):
    parData = pd.DataFrame()
    manualData = pd.DataFrame()
    lastTimepoint, lastTimepoint2 = [], []    # the last time points in each situation

    for s in range(1,10):
        # participants' ratings
        parDat = pd.read_csv(join(parDataDir, f"narrative{n}situation{s}.csv"))
        if s != 1:
            parDat['time'] += 0.23
            parDat['time'] += lastTimepoint[-1]
        lastTimepoint.append(parDat.loc[len(parDat)-1, 'time'])
        
        parDat = parDat[~parDat['ID'].isin(badSub)]    # excluding subjects that are not like the group
        
        parData = pd.concat([parData, parDat], ignore_index=True)
        parData = parData.drop(columns='index') if s!=1 else parData
        parData = parData.reset_index()

        # manual ratings
        manualDat = pd.read_csv(join(manualDataDir, f"narrative{n}situation{s}.csv"))
        if s != 1:
            manualDat['time'] += 0.23
            manualDat['time'] += lastTimepoint2[-1]
        lastTimepoint2.append(manualDat.loc[len(manualDat)-1, 'time'])
        manualDat['situation'] = s
        manualDat['validPar_trial'] = len(np.unique(parDat['ID']))     # the number of participants who have valid data in this trial
        
        nvalid_time = parDat.groupby('time').count()    # the number of participants who have valid data at each time point
        nvalid_time = nvalid_time.reset_index()
        manualDat['validPar_time'] = nvalid_time['rating']
        
        manualData = pd.concat([manualData, manualDat], ignore_index=True)
        manualData = manualData.drop(columns='index') if s!= 1 else manualData
        manualData = manualData.reset_index()

    manualData = manualData.drop(columns='index')
    manualData.columns = ['time', 'labels', 'situation', 'validPar_trial', 'validPar_time']
    manualData = manualData[['situation', 'time', 'labels', 'validPar_trial', 'validPar_time']]
    
    parData_mean = parData.groupby('time')['rating'].mean()
    parData_mean = parData_mean.reset_index()
    parData_mean = parData_mean.drop(columns='time')
    parData_mean.columns = ['mean']
    parData_std = parData.groupby('time')['rating'].std()
    parData_std = parData_std.reset_index()
    parData_std = parData_std.drop(columns='time')
    parData_std.columns = ['std']
    
    combinedData = pd.concat([manualData, parData_mean, parData_std], axis=1)
    combinedData.to_csv(join(dataDir, 'combined_tom_zizhuang', f'narrative{n}_mean.csv'), index=False)
    allData = pd.concat([allData, combinedData], ignore_index=True)

    # calculate metrics
    # correlation
    corr_narrative.append(combinedData['labels'].corr(combinedData['mean']))
    # logistic regression
    binLabels = pd.DataFrame({'labels': [1 if x > 50 else 0 for x in combinedData['labels']]})
    X = combinedData['mean']
    binLabels = binLabels.loc[~X.isna(), 'labels']
    X = sm.add_constant(X)
    X = X.dropna()
    logit_model = sm.Logit(binLabels, X)
    result = logit_model.fit()
    regress_pRsquare_narrative.append(result.prsquared)
    regress_z_narrative.append(result.tvalues[1])
    # t test
    t_stat, p_val = ttest_ind(binLabels, X.iloc[:, 1])
    t_narratvie.append(t_stat)
    p_t_narrative.append(p_val)

# calculate metrics for all narratives concatenated
# correlation
corr_narrative.append(allData['labels'].corr(allData['mean']))
# logistic regression
binLabels = pd.DataFrame({'labels': [1 if x > 50 else 0 for x in allData['labels']]})
X = allData['mean']
binLabels = binLabels.loc[~X.isna(), 'labels']
X = sm.add_constant(X)
X = X.dropna()
logit_model = sm.Logit(binLabels, X)
result = logit_model.fit()
regress_pRsquare_narrative.append(result.prsquared)
regress_z_narrative.append(result.tvalues[1])
# t test
t_stat, p_val = ttest_ind(binLabels, X.iloc[:, 1])
t_narratvie.append(t_stat)
p_t_narrative.append(p_val)


outputDf_narrative = pd.DataFrame({'narrative': list(range(1,9)) + ['All'], 
                                   'pearson_r': corr_narrative, 't': t_narratvie, 'p_ttest': p_t_narrative,
                                   'z_logistic': regress_z_narrative, 'pseudo_r': regress_pRsquare_narrative})
outputDf_narrative = outputDf_narrative.round(3)
#outputDf_narrative.to_csv(join(outputDir, 'similarity_con_tom_1a2a3a4a5a.csv'), index=False)

Optimization terminated successfully.
         Current function value: 0.635329
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.689536
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.648266
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.655832
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.648446
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.628600
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.651065
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.669272
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.667591
         Iterations 5


##### median

In [7]:
from scipy.stats import median_abs_deviation

dataDir = 'C:\\'
parDataDir = join(dataDir, 'continuous_tom_1a2a3a4a_long')
manualDataDir = join(dataDir, 'manualAnnotations_tom_zizhuang')
outputDir = 'C:\\'

badSub = pd.read_csv('C:\\')['ID']

corr_narrative = []
regress_pRsquare_narrative = []
regress_z_narrative = []
t_narratvie = []
p_t_narrative = []

allData = pd.DataFrame()

for n in range(1,9):
    parData = pd.DataFrame()
    manualData = pd.DataFrame()
    lastTimepoint, lastTimepoint2 = [], []    # the last time points in each situation

    for s in range(1,10):
        # participants' ratings
        parDat = pd.read_csv(join(parDataDir, f"narrative{n}situation{s}.csv"))
        if s != 1:
            parDat['time'] += 0.23
            parDat['time'] += lastTimepoint[-1]
        lastTimepoint.append(parDat.loc[len(parDat)-1, 'time'])
        parDat = parDat[~parDat['ID'].isin(badSub)]    # excluding subjects that are not like the group
        parData = pd.concat([parData, parDat])
        parData = parData.drop(columns='index') if s!=1 else parData
        parData = parData.reset_index()

        # manual ratings
        manualDat = pd.read_csv(join(manualDataDir, f"narrative{n}situation{s}.csv"))
        if s != 1:
            manualDat['time'] += 0.23
            manualDat['time'] += lastTimepoint2[-1]
        lastTimepoint2.append(manualDat.loc[len(manualDat)-1, 'time'])
        manualDat['situation'] = s
        manualDat['validPar_trial'] = len(np.unique(parDat['ID']))     # the number of participants who have valid data in this trial
        nvalid_time = parDat.groupby('time').count()    # the number of participants who have valid data at each time point
        nvalid_time = nvalid_time.reset_index()
        manualDat['validPar_time'] = nvalid_time['rating']
        manualData = pd.concat([manualData, manualDat])
        manualData = manualData.drop(columns='index') if s!= 1 else manualData
        manualData = manualData.reset_index()

    manualData = manualData.drop(columns='index')
    manualData.columns = ['time', 'labels', 'situation', 'validPar_trial', 'validPar_time']
    manualData = manualData[['situation', 'time', 'labels', 'validPar_trial', 'validPar_time']]
    
    parData_median = parData.groupby('time')['rating'].median()
    parData_median = parData_median.reset_index()
    parData_median = parData_median.drop(columns='time')
    parData_median.columns = ['median']
    parData_mad = parData.dropna().groupby('time')['rating'].apply(median_abs_deviation)
    parData_mad = parData_mad.reset_index()
    parData_mad = parData_mad.drop(columns='time')
    parData_mad.columns = ['mad']
    
    combinedData = pd.concat([manualData, parData_median, parData_mad], axis=1)
    combinedData.to_csv(join(dataDir, 'combined_tom_zizhuang', f'narrative{n}_median.csv'), index=False)
    allData = pd.concat([allData, combinedData], ignore_index=True)

    # calculate metrics
    # correlation
    corr_narrative.append(combinedData['labels'].corr(combinedData['median']))
    # logistic regression
    binLabels = pd.DataFrame({'labels': [1 if x > 50 else 0 for x in combinedData['labels']]})
    X = combinedData['median']
    binLabels = binLabels.loc[~X.isna(), 'labels']
    X = sm.add_constant(X)
    X = X.dropna()
    logit_model = sm.Logit(binLabels, X)
    result = logit_model.fit()
    regress_pRsquare_narrative.append(result.prsquared)
    regress_z_narrative.append(result.tvalues[1])
    # t test
    t_stat, p_val = ttest_ind(binLabels, X.iloc[:, 1])
    t_narratvie.append(t_stat)
    p_t_narrative.append(p_val)

# calculate metrics for all narratives concatenated
# correlation
corr_narrative.append(allData['labels'].corr(allData['median']))
# logistic regression
binLabels = pd.DataFrame({'labels': [1 if x > 50 else 0 for x in allData['labels']]})
X = allData['median']
binLabels = binLabels.loc[~X.isna(), 'labels']
X = sm.add_constant(X)
X = X.dropna()
logit_model = sm.Logit(binLabels, X)
result = logit_model.fit()
regress_pRsquare_narrative.append(result.prsquared)
regress_z_narrative.append(result.tvalues[1])
# t test
t_stat, p_val = ttest_ind(binLabels, X.iloc[:, 1])
t_narratvie.append(t_stat)
p_t_narrative.append(p_val)


outputDf_narrative = pd.DataFrame({'narrative': list(range(1,9)) + ['All'], 
                                   'pearson_r': corr_narrative, 't': t_narratvie, 'p_ttest': p_t_narrative,
                                   'z_logistic': regress_z_narrative, 'pseudo_r': regress_pRsquare_narrative})
outputDf_narrative = outputDf_narrative.round(3)
outputDf_narrative.to_csv(join(outputDir, 'similarity_tom_conti_v_zizhuang_median.csv'), index=False)

Optimization terminated successfully.
         Current function value: 0.634454
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.688505
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.647223
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.655465
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.656165
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.642057
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.653317
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.669391
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.668375
         Iterations 5


##### **Calculate similarity between ToM and ToM demands by consensus labels**

In [8]:
from scipy.stats import median_abs_deviation

dataDir = 'C:\\'
parDataDir = join(dataDir, 'continuous_tom_1a2a3a4a_long')
manualDataDir = join(dataDir, 'manualAnnotations_tom_demands_consensus')
outputDir = 'C:\\'

badSub = pd.read_csv('C:\\')['ID']

corr_narrative = []
regress_pRsquare_narrative = []
regress_z_narrative = []
t_narratvie = []
p_t_narrative = []

allData = pd.DataFrame()

for n in range(1,9):
    parData = pd.DataFrame()
    manualData = pd.DataFrame()
    lastTimepoint, lastTimepoint2 = [], []    # the last time points in each situation

    for s in range(1,10):
        # participants' ratings
        parDat = pd.read_csv(join(parDataDir, f"narrative{n}situation{s}.csv"))
        if s != 1:
            parDat['time'] += 0.23
            parDat['time'] += lastTimepoint[-1]
        lastTimepoint.append(parDat.loc[len(parDat)-1, 'time'])
        parDat = parDat[~parDat['ID'].isin(badSub)]    # excluding subjects that are not like the group
        parData = pd.concat([parData, parDat])
        parData = parData.drop(columns='index') if s!=1 else parData
        parData = parData.reset_index()

        # manual ratings
        manualDat = pd.read_csv(join(manualDataDir, f"narrative{n}situation{s}.csv"))
        if s != 1:
            manualDat['time'] += 0.23
            manualDat['time'] += lastTimepoint2[-1]
        lastTimepoint2.append(manualDat.loc[len(manualDat)-1, 'time'])
        manualDat['situation'] = s
        manualDat['validPar_trial'] = len(np.unique(parDat['ID']))     # the number of participants who have valid data in this trial
        nvalid_time = parDat.groupby('time').count()    # the number of participants who have valid data at each time point
        nvalid_time = nvalid_time.reset_index()
        manualDat['validPar_time'] = nvalid_time['rating']
        manualData = pd.concat([manualData, manualDat])
        manualData = manualData.drop(columns='index') if s!= 1 else manualData
        manualData = manualData.reset_index()

    manualData = manualData.drop(columns='index')
    manualData.columns = ['time', 'labels', 'situation', 'validPar_trial', 'validPar_time']
    manualData = manualData[['situation', 'time', 'labels', 'validPar_trial', 'validPar_time']]
    
    parData_median = parData.groupby('time')['rating'].median()
    parData_median = parData_median.reset_index()
    parData_median = parData_median.drop(columns='time')
    parData_median.columns = ['median']
    parData_mad = parData.dropna().groupby('time')['rating'].apply(median_abs_deviation)
    parData_mad = parData_mad.reset_index()
    parData_mad = parData_mad.drop(columns='time')
    parData_mad.columns = ['mad']
    
    combinedData = pd.concat([manualData, parData_median, parData_mad], axis=1)
    combinedData.to_csv(join(dataDir, 'combined_tom_consensus', f'narrative{n}_median.csv'), index=False)
    allData = pd.concat([allData, combinedData], ignore_index=True)

    # calculate metrics
    # correlation
    corr_narrative.append(combinedData['labels'].corr(combinedData['median']))
    # logistic regression
    binLabels = pd.DataFrame({'labels': [1 if x > 50 else 0 for x in combinedData['labels']]})
    X = combinedData['median']
    binLabels = binLabels.loc[~X.isna(), 'labels']
    X = sm.add_constant(X)
    X = X.dropna()
    logit_model = sm.Logit(binLabels, X)
    result = logit_model.fit()
    regress_pRsquare_narrative.append(result.prsquared)
    regress_z_narrative.append(result.tvalues[1])
    # t test
    t_stat, p_val = ttest_ind(binLabels, X.iloc[:, 1])
    t_narratvie.append(t_stat)
    p_t_narrative.append(p_val)

# calculate metrics for all narratives concatenated
# correlation
corr_narrative.append(allData['labels'].corr(allData['median']))
# logistic regression
binLabels = pd.DataFrame({'labels': [1 if x > 50 else 0 for x in allData['labels']]})
X = allData['median']
binLabels = binLabels.loc[~X.isna(), 'labels']
X = sm.add_constant(X)
X = X.dropna()
logit_model = sm.Logit(binLabels, X)
result = logit_model.fit()
regress_pRsquare_narrative.append(result.prsquared)
regress_z_narrative.append(result.tvalues[1])
# t test
t_stat, p_val = ttest_ind(binLabels, X.iloc[:, 1])
t_narratvie.append(t_stat)
p_t_narrative.append(p_val)


outputDf_narrative = pd.DataFrame({'narrative': list(range(1,9)) + ['All'], 
                                   'pearson_r': corr_narrative, 't': t_narratvie, 'p_ttest': p_t_narrative,
                                   'z_logistic': regress_z_narrative, 'pseudo_r': regress_pRsquare_narrative})
outputDf_narrative = outputDf_narrative.round(3)
outputDf_narrative.to_csv(join(outputDir, 'similarity_tom_conti_v_cons_median.csv'), index=False)

Optimization terminated successfully.
         Current function value: 0.662772
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.683457
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.667131
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.685041
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.599663
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.669779
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.666869
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.668652
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.675817
         Iterations 4
